In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('Time Series.xlsx')

In [3]:
df.shape

(10885, 4)

In [4]:
activity_counts = df['Activity'].value_counts()
len(activity_counts)

10

In [5]:
df['Check'] = df.apply(lambda row: 'yes' if row['Start'] < row['End'] else 'no', axis=1)

In [6]:
df['Check'].unique()

array(['no', 'yes'], dtype=object)

In [7]:
check_counts = df['Check'].value_counts()
check_counts

Check
no     5478
yes    5407
Name: count, dtype: int64

In [22]:
df['Start'].unique

<bound method Series.unique of 4341    2023-03-29 15:31:52.620
10807   2023-03-29 15:31:52.620
10356   2023-03-29 15:31:52.620
10846   2023-03-29 17:55:52.620
9894    2023-03-29 17:55:52.620
                  ...          
1542    2023-10-05 05:55:52.620
3899    2023-10-06 01:07:52.620
8894    2023-10-06 13:07:52.620
8103    2023-10-07 10:43:52.620
1567    2023-10-09 22:43:52.620
Name: Start, Length: 10885, dtype: datetime64[ns]>

In [8]:
condition = df['Start'] > df['End']
df.loc[condition, ['Start', 'End']] = df.loc[condition, ['End', 'Start']].values


In [9]:
df = df.sort_values(by=['Name','Start','End'])

df['prev_end_time'] = df.groupby('Name')['End'].shift(1)

df['group_id'] = np.where((df['prev_end_time'].isnull()) | (df['Start'] > df['prev_end_time']), 1, 0)

df['period_number'] = df.groupby('Name')['group_id'].cumsum()



In [10]:
df['period_number']

4341     1
10807    1
10356    1
10846    1
9894     1
        ..
1542     3
3899     3
8894     4
8103     4
1567     4
Name: period_number, Length: 10885, dtype: int32

In [24]:
aggregated = df.groupby(['Name', 'period_number']).agg(
    period_start=('Start', 'min'),
    period_end=('End', 'max'),
    activities=('Activity', lambda x: ', '.join(x))
).reset_index()

aggregated = aggregated.sort_values(by=['period_start','Name'])

In [25]:
aggregated.reset_index(drop= True, inplace = True)
aggregated.drop(columns='period_number', inplace= True)

In [26]:
aggregated

,Name,period_start,period_end,activities
0,Deepti,2023-03-29 15:31:52.620,2023-10-15 08:19:52.620,"Business Development, Reply to Customers, Podc..."
1,Jyoti,2023-03-29 15:31:52.620,2023-10-13 13:07:52.620,"Business Development, Updates, Fund raising, I..."
2,Priyanka,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"Business Development, Reply to Customers, Remo..."
3,Ravi,2023-03-29 15:31:52.620,2023-10-15 13:07:52.620,"Call, Remote Inspection, Call, Remote Inspecti..."
4,Sharan,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"Inspection, Updates, Fund raising, Send Email,..."
5,Deepti,2023-04-02 15:31:52.620,2023-10-15 13:07:52.620,"Reply to Customers, Business Development, Upda..."
6,Jyoti,2023-04-10 08:19:52.620,2023-10-14 15:31:52.620,"Business Development, Remote Inspection, Podca..."
7,Jyoti,2023-05-09 01:07:52.620,2023-10-15 13:07:52.620,"Updates, Business Development, Call, Inspectio..."
8,Deepti,2023-09-06 20:19:52.620,2023-10-15 15:31:52.620,"Reply to Customers, Inspection, Remote Inspect..."
9,Priyanka,2023-09-06 22:43:52.620,2023-10-15 13:07:52.620,"Fund raising, Business Development, Business D..."
